In [ ]:
import openai
import datetime
import json
import random
import pandas as pd
from docx import Document
from docx.shared import Pt
from textblob import TextBlob
import pickle

# API Key for OpenAI
API_KEY = "Enter Your Key"
openai.api_key = API_KEY

#This is to limit the channels which the system will generate the output for, the user will have to select which ones they need
campaign_channels = ["Facebook", "Instagram", "Google", "LinkedIn"]

categorized_activities = {
    "Brand Awareness": ["Introduce the brand’s mission and values", "Share a founder’s story", "Explain the product's unique selling points", "Publish an industry trend analysis"],
    "Engagement & Interaction": ["Run a poll to gather audience insights", "Host a live Q&A session", "Encourage user-generated content", "Start a viral challenge or hashtag"],
    "Product Highlights": ["Showcase a product demo", "Highlight a key feature with a visual post", "Compare with competitors’ offerings", "Announce an upcoming product launch"],
    "Promotions & Offers": ["Announce a limited-time discount", "Share a referral program incentive", "Create a bundled offer post", "Host a giveaway contest"],
    "Customer Testimonials": ["Post a customer success story", "Share a video testimonial", "Highlight a 5-star review", "Feature an influencer's opinion"],
    "Campaign Wrap-up & Retargeting": ["Summarize campaign performance", "Share behind-the-scenes content", "Retarget customers with special offers", "Announce future plans based on insights"]
}

weekly_themes = list(categorized_activities.keys())

#Get the users inputs to define the parameters
#Includes product, audience, chosen platforms, duration, and total budget
def get_user_inputs():
    while True:
        try:
            product = input("Enter the product/service name: ")
            audience = input("Describe the target audience (e.g., age, interests, location): ")

            print("Select one or more Campaign Channels (1. Facebook, 2. Instagram, 3. Google, 4. LinkedIn):")
            for i, ch in enumerate(campaign_channels, 1):
                print(f"{i}. {ch}")
            ch_input = input("Your choices (comma-separated): ")

            selected_channels = [campaign_channels[int(i.strip()) - 1] for i in ch_input.split(',') if i.strip().isdigit()]

            campaign_duration = int(input("Enter campaign duration (in weeks): "))
            total_budget = float(input("Enter total budget for the campaign £: "))

            return {
                "product": product,
                "audience": audience,
                "platforms": selected_channels,
                "campaign_duration": campaign_duration,
                "total_budget": total_budget
            }

        except (ValueError, IndexError):
            print("\n Invalid input. Please try again with correct values.\n")

#This Constructs a prompt and uses OpenAI API to generate a marketing strategy
#Includes messaging, segmentation, competitor analysis, and weekly breakdown
def generate_strategy(product, audience, platforms, campaign_duration, total_budget):
    prompt = f"""Generate a detailed marketing strategy for a {product} campaign.
    Target Audience: {audience}
    Platforms: {', '.join(platforms)}
    Campaign Duration: {campaign_duration} weeks
    Total Budget: £{total_budget}
    Include messaging, segmentation, competitor analysis, and a weekly breakdown strictly aligned to these constraints."""

    try:
        response = openai.ChatCompletion.create(
            model="gpt-4o",
            temperature=1.0,
            messages=[
                {"role": "system", "content": "You are a marketing strategist."},
                {"role": "user", "content": prompt}
            ]
        )
        content = response['choices'][0]['message']['content']
    except Exception as e:
        print(f"\nError generating strategy from OpenAI: {e}")
        return "Strategy generation failed. Please check your API connection or try again later."

    for symbol in ["**", "##", "###"]:
        content = content.replace(symbol, "")
    content += "\n\n--- Competitor Analysis ---\n"
    content += "Top competitors for this campaign include well-known brands in the space.\n"
    content += "Reviewing content from Meta Ad Library or Google Ads Preview suggests strong emphasis on visual storytelling and influencer-driven messaging.\n"
    content += "Key takeaway: focus on authentic engagement and highlight value propositions clearly."

    return content

#Generates ad copy and creative concepts for selected platforms
#Performs readability and sentiment analysis, and retries if content is too complex    
def generate_creatives(product, platforms):
    import textstat
    creative_outputs = []
    for platform in platforms:
        if platform.lower() in ["facebook", "instagram", "linkedin"]:
            ad_prompt = f"""
            Write the following for a {product} campaign on {platform}:
            - A compelling ad headline
            - A persuasive ad description
            - A description of the creative concept (what the image or video should look like)
            - Include a list of 10 relevant hashtags (no headings)
            """
        elif platform.lower() == "google":
            ad_prompt = f"""
            Generate Google Search ad copy for a {product} campaign. Include clearly labeled sections:

            Headline 1:
            Headline 2:
            Headline 3:

            Description 1:
            Description 2:

            Keywords:
            Provide a list of 10 relevant keywords.
            """
        else:
            continue

        response = openai.ChatCompletion.create(
            model="gpt-4o",
            temperature=1.2,
            messages=[
                {"role": "system", "content": "You are a creative advertising expert."},
                {"role": "user", "content": ad_prompt.strip()}
            ]
        )
        content = response['choices'][0]['message']['content']

        for phrase in ["**Ad Description:**", "**Ad Headline:**", "**Creative Concept:**",
                       "**Headlines:**", "**Descriptions:**", "**Keywords:**", "**Hashtags:**"]:
            content = content.replace(phrase, "")

        readability = textstat.flesch_reading_ease(content)
        grade_level = textstat.flesch_kincaid_grade(content)

        if readability < 50:
            retry_prompt = f"""Please simplify the following ad content for a {platform} campaign. Use plain language and short sentences:\n\n{content}"""
            retry_response = openai.ChatCompletion.create(
                model="gpt-4o",
                temperature=1.2,
                messages=[
                    {"role": "system", "content": "You simplify marketing content for general audiences."},
                    {"role": "user", "content": retry_prompt.strip()}
                ]
            )
            content = retry_response['choices'][0]['message']['content']
            readability = textstat.flesch_reading_ease(content)
            grade_level = textstat.flesch_kincaid_grade(content)

        if readability >= 80:
            summary = "Very easy to read. Suitable for a wide audience."
        elif readability >= 60:
            summary = "Quite readable. Should be easy for most consumers to understand."
        elif readability >= 40:
            summary = "Moderately difficult. May require simplification for broader reach."
        else:
            summary = "Difficult to read. Consider rewriting for better accessibility."

        content += (
            f"\n\n--- Readability Analysis ---"
            f"\nFlesch Reading Ease Score: {readability:.2f}"
            f"\nFlesch-Kincaid Grade Level: {grade_level:.2f}"
            f"\nSummary: {summary}"
        )

        blob = TextBlob(content)
        sentiment = blob.sentiment.polarity
        sentiment_summary = "Positive" if sentiment > 0.2 else "Neutral" if sentiment >= -0.1 else "Negative"
        content += (
            f"\n\n--- Sentiment Analysis ---"
            f"\nSentiment Score: {sentiment:.2f}"
            f"\nSentiment Summary: {sentiment_summary}"
        )

        creative_outputs.append((platform, content))

    return creative_outputs

#This generates the campaign calender which is then saved as a excel spreadsheet
def generate_campaign_calendar(start_date, weeks, platforms, total_budget):
    calendar = []
    current_date = datetime.datetime.strptime(start_date, "%Y-%m-%d")
    days = weeks * 7

    try:
        with open("platform_weights.pkl", "rb") as f:
            priority_weights = pickle.load(f)
    except FileNotFoundError:
        print("⚠️  platform_weights.pkl not found. Using default weights.")
        priority_weights = {"google": 0.4, "facebook": 0.3, "instagram": 0.2, "linkedin": 0.1}

    total_weight = sum([priority_weights.get(p.lower(), 0) for p in platforms])

    platform_daily_budget = {
        p.lower(): (total_budget * priority_weights.get(p.lower(), 0) / total_weight) / days if total_weight > 0 else 0
        for p in platforms
    }

    for week in range(weeks):
        theme = weekly_themes[week % len(weekly_themes)]
        for day in range(7):
            content = random.choice(categorized_activities[theme])
            content_type = "Static" if day % 2 == 0 else "Dynamic"
            weekday = current_date.strftime("%A")

            for platform in platforms:
                platform_key = platform.lower()
                budget = round(platform_daily_budget.get(platform_key, 0), 2)
                calendar.append({
                    "Date": current_date.strftime("%d.%m.%y"),
                    "Day": weekday,
                    "Content": content,
                    "Content Type": content_type,
                    "Platform": platform,
                    "Theme": theme,
                    "Budget Allocation (£)": budget
                })

            current_date += datetime.timedelta(days=1)

    return calendar

#This creates the word report which stores our marketing strategy, competitor analysis, content for different platforms, week wise breakdown and creative ideation
def create_word_report(user_inputs, strategy, creatives):
    doc = Document()
    doc.add_heading("Marketing Campaign Strategy Report", level=1)

    doc.add_heading("Campaign Overview", level=2)
    for key in ["product", "audience", "platforms", "campaign_duration", "total_budget"]:
        p = doc.add_paragraph(f"{key.replace('_', ' ').title()}: {user_inputs[key]}")
        p.runs[0].font.name = "Times New Roman"
        p.runs[0].font.size = Pt(11)

    doc.add_heading("Strategy", level=2)
    p = doc.add_paragraph(strategy)
    p.runs[0].font.name = "Times New Roman"
    p.runs[0].font.size = Pt(11)

    doc.add_heading("Ad Copy & Creative Ideation", level=2)
    for platform, content in creatives:
        doc.add_heading(platform, level=3)
        p = doc.add_paragraph(content)
        p.runs[0].font.name = "Times New Roman"
        p.runs[0].font.size = Pt(11)

    path = r"WORD FILE PATH"
    doc.save(path)
    print("\n--- Strategy Word Report saved at:", path)

#This is the main workflow function: orchestrates input collection, strategy generation, creative generation, campaign calendar creation, and report output
def main():
    user_inputs = get_user_inputs()
    strategy = generate_strategy(
        product=user_inputs["product"],
        audience=user_inputs["audience"],
        platforms=user_inputs["platforms"],
        campaign_duration=user_inputs["campaign_duration"],
        total_budget=user_inputs["total_budget"]
    )
    creatives = generate_creatives(user_inputs["product"], user_inputs["platforms"])
    calendar = generate_campaign_calendar(
        start_date=datetime.date.today().strftime("%Y-%m-%d"),
        weeks=user_inputs['campaign_duration'],
        platforms=user_inputs['platforms'],
        total_budget=user_inputs['total_budget']
    )

    df = pd.DataFrame(calendar)
    output_path = r"EXCEL FILE PATH"
    df.to_excel(output_path, index=False)

    print("\n--- Marketing Strategy ---\n")
    print(strategy)
    print("\n--- Campaign Calendar saved at:", output_path)

    create_word_report(user_inputs, strategy, creatives)

if __name__ == "__main__":
    main()
